# Training Pipeline

All dataloading and training in one place!

In [1]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)
import torch
import torch.nn as nn
import torch.nn.functional as F
import torchvision.transforms as transforms
from torch.utils.data import Dataset, DataLoader
import numpy as np
from sklearn.model_selection import KFold, GroupKFold, train_test_split
from datetime import datetime
from tqdm.notebook import tqdm
import albumentations as A
import matplotlib.pyplot as plt
import os
from typing import List, Tuple, Union
import pickle
from lightning.pytorch.loggers.tensorboard import TensorBoardLogger
from torcheval.metrics import MulticlassAccuracy
from torcheval.metrics.functional import multiclass_f1_score

#Local packages
from src import CNNDetector, ContrastiveDetector

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

# for dirname, _, filenames in os.walk('/kaggle/input'):
#     for filename in filenames:
#         print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

In [2]:
"""
Configure

Change the paths of the data directory to the location of your HMS Dataset.
The sub directories of TRAIN/TEST_EEG, TRAIN/TEST_SPEC should remain the same

"""

#Kaggle
# DATA_ROOT = "/kaggle/input/hms-harmful-brain-activity-classification/"
# TRAIN_EEG = "train_eegs"
# TRAIN_SPEC = "train_spectrograms"
# TEST_EEG = "test_eegs"
# TEST_SPEC = "test_spectrograms"

#Local
DATA_ROOT = "/home/benluo/HBAC/data/hbac"
TRAIN_EEG = "train_eegs"
TRAIN_SPEC = "train_spectrograms"
TEST_EEG = "test_eegs"
TEST_SPEC = "test_spectrograms"

#Other hyperparams
VALID_FOLDS = 5

In [3]:
device = "cuda" if torch.cuda.is_available() else "cpu"
print(device)

cuda


### Reduce train dataset size

The code below only takes the first portion of data in the training list to reduce the number of spectrogram and EEG samples in the training dataset

You can choose not to run the following cell if `train_reduced.csv` already exists

In [4]:
#Get reduced train list as the entire dataset is heavy on RAM, 

reduce_train_path = os.path.join(DATA_ROOT, "train_reduced.csv")

if os.path.isfile(reduce_train_path):

    print("train_reduced.csv already exists")

else:

    train_list = pd.read_csv(os.path.join(DATA_ROOT, "train.csv"))

    reduced_len = train_list.shape[0]//2 #32 GB of RAM during training
    # reduced_len = train_list.shape[0]//4 #Less RAM but less data

    train_list_reduced = train_list.iloc[:reduced_len,:]

    train_list_reduced.to_csv(os.path.join(DATA_ROOT, "train_reduced.csv"))

train_reduced.csv already exists


### Generate Data

Since test data does not come with labels, for local training and testing, we will only use the data from `train.csv`

The Dataset follows a train-val-test split.

The train and test datasets are split first by `test_size`.

The train and validation datasets are split by Group K-Folds into `val_folds` uniform groups. One group will be chosen randomly for validation while the rest will be used for training every epoch.

In [5]:
def generate_data(val_folds:int = VALID_FOLDS,
                  test_size:float = 0.1,
                  saved_spec:str = "all_spec.pkl",
                  saved_eeg:str = "all_eeg.pkl"):

    #Use reduced csv file
    train_list = pd.read_csv(os.path.join(DATA_ROOT, "train_reduced.csv"))

    print("All data", train_list.shape)
    display(train_list.head())

    label_cols = train_list.columns[-6:]

    #Create new df to be formated for training and testing
    train_df = train_list[['spectrogram_id','eeg_id','patient_id','spectrogram_label_offset_seconds','eeg_label_offset_seconds']]

    #Normalise labels into probabilities
    aux = train_list[label_cols].copy()
    
    for label in label_cols:
        train_df[label] = aux[label].values
        
    y_data = train_df[label_cols].values
    y_data = y_data / y_data.sum(axis=1,keepdims=True)
    train_df[label_cols] = y_data

    #Target label/class
    aux = train_list['expert_consensus'].copy()
    train_df['target'] = aux
    train_df = train_df.reset_index()

    #Sort df by patient id so testing and training data will be less similar
    train_df = train_df.sort_values("patient_id")

    test_df = train_df.iloc[int((1-test_size)*train_df.shape[0]):]

    train_df = train_df.iloc[:int((1-test_size)*train_df.shape[0])]

    train_df = train_df.reset_index()
    test_df = test_df.reset_index()

    print("Training data")
    display(train_df.head())

    print("Testing data")
    display(test_df.head())


    gkf = GroupKFold(n_splits=val_folds)

    #KFold grouped by patient id
    for fold, (train_index, val_index) in enumerate(gkf.split(train_df, train_df.target, train_df.patient_id)):
        train_df.loc[val_index, "fold"] = int(fold)
    
    all_eeg = {}
    
    all_spec = {}

    #Try loading eeg and spec data if they have been generated previously
    #If not, read eeg and spec data from train_list and save the collection in data folder
    #Data is saved as float16 due to space constraints
    try:

        with open(os.path.join(DATA_ROOT,saved_spec), "rb") as handle:
            all_spec = pickle.load(handle)

        with open(os.path.join(DATA_ROOT,saved_eeg), "rb") as handle:
            all_eeg = pickle.load(handle)

    except:
    
        for idx, row in tqdm(train_list.iterrows()):

            spec_id = row["spectrogram_id"]
            eeg_id = row["eeg_id"]

            if spec_id not in all_spec:

                spec = pd.read_parquet(os.path.join(DATA_ROOT, TRAIN_SPEC, str(spec_id)+".parquet"))

                all_spec[spec_id] = spec.iloc[:,1:].values.astype(dtype=np.float32)

            if eeg_id not in all_eeg:

                eeg = pd.read_parquet(os.path.join(DATA_ROOT, TRAIN_EEG, str(eeg_id)+".parquet"))

                all_eeg[eeg_id] = eeg.iloc[:,1:].values.astype(dtype=np.float32)
        
        with open(os.path.join(DATA_ROOT, saved_eeg), "wb") as handle:
            pickle.dump(all_eeg, handle)
        
        with open(os.path.join(DATA_ROOT, saved_spec), "wb") as handle:
            pickle.dump(all_spec, handle)
    
    return train_df, test_df, all_eeg, all_spec, label_cols

In [6]:
train_df, test_df, all_eeg, all_spec, label_cols = generate_data()

All data (53400, 16)


,Unnamed: 0,eeg_id,eeg_sub_id,eeg_label_offset_seconds,spectrogram_id,spectrogram_sub_id,spectrogram_label_offset_seconds,label_id,patient_id,expert_consensus,seizure_vote,lpd_vote,gpd_vote,lrda_vote,grda_vote,other_vote
0,0,1628180742,0,0.0,353733,0,0.0,127492639,42516,Seizure,3,0,0,0,0,0
1,1,1628180742,1,6.0,353733,1,6.0,3887563113,42516,Seizure,3,0,0,0,0,0
2,2,1628180742,2,8.0,353733,2,8.0,1142670488,42516,Seizure,3,0,0,0,0,0
3,3,1628180742,3,18.0,353733,3,18.0,2718991173,42516,Seizure,3,0,0,0,0,0
4,4,1628180742,4,24.0,353733,4,24.0,3080632009,42516,Seizure,3,0,0,0,0,0


Training data


/home/benluo/miniconda3/envs/DeepfakeBench/lib/python3.7/site-packages/ipykernel_launcher.py:21: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
/home/benluo/miniconda3/envs/DeepfakeBench/lib/python3.7/site-packages/pandas/core/frame.py:3678: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  self[col] = igetitem(value, i)
/home/benluo/miniconda3/envs/DeepfakeBench/lib/python3.7/site-packages/ipykernel_launcher.py:29: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try usi

,level_0,index,spectrogram_id,eeg_id,patient_id,spectrogram_label_offset_seconds,eeg_label_offset_seconds,seizure_vote,lpd_vote,gpd_vote,lrda_vote,grda_vote,other_vote,target
0,41744,41744,802850878,1873660287,56,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,Other
1,49045,49045,957002006,165634434,56,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,Other
2,41746,41746,802850878,2057577408,56,290.0,46.0,0.0,0.0,0.0,0.0,0.0,1.0,Other
3,41745,41745,802850878,2057577408,56,244.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,Other
4,25278,25278,497667405,374550767,56,694.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,Other


Testing data


,level_0,index,spectrogram_id,eeg_id,patient_id,spectrogram_label_offset_seconds,eeg_label_offset_seconds,seizure_vote,lpd_vote,gpd_vote,lrda_vote,grda_vote,other_vote,target
0,5410,5410,91996359,3686473557,57272,122.0,122.0,0.0,0.0,0.0,1.0,0.0,0.0,LRDA
1,5389,5389,91996359,3686473557,57272,72.0,72.0,0.0,0.0,0.0,1.0,0.0,0.0,LRDA
2,5384,5384,91996359,3686473557,57272,60.0,60.0,0.0,0.0,0.0,1.0,0.0,0.0,LRDA
3,5394,5394,91996359,3686473557,57272,82.0,82.0,0.0,0.0,0.0,1.0,0.0,0.0,LRDA
4,5385,5385,91996359,3686473557,57272,62.0,62.0,0.0,0.0,0.0,1.0,0.0,0.0,LRDA


In [7]:
class HMSDataset(Dataset):

    def __init__(self,
                 df:pd.DataFrame = None,
                 aug: bool = False) -> None:
        

        super(HMSDataset, self).__init__()

        self.df = df

        self.eeg_sample_freq = 200 # 200 Hz
        self.spec_sample_freq = 0.5 # 0.5 Hz
    
        
        #data augmentation
        self.aug = aug
        self.transforms = A.Compose([
            A.HorizontalFlip(p=0.5)
        ])

    def format_data(self, eeg:np.array, spec:np.array) -> Tuple[torch.tensor]:

        #Epsilon for numerical stability during division (prevent division by zero)
        eps = 1e-6

        #Convert data from saved float16 to float32 during training and testing
        eeg = eeg.astype(dtype=np.float32)
        spec = spec.astype(dtype=np.float32)

        #Normalising and getting rid of Nans
        eeg_mean = np.nanmean(eeg.flatten())
        eeg_std = np.nanstd(eeg.flatten())
        eeg = (eeg-eeg_mean)/(eeg_std+eps)
        eeg = np.nan_to_num(eeg, nan=0.0)

        #Limiting range of spec data
        spec = np.clip(spec, np.exp(-4), np.exp(8))
        spec = np.log(spec)
        
        #Normalising and getting rid of Nans
        spec_mean = np.nanmean(spec.flatten())
        spec_std = np.nanstd(spec.flatten())
        spec = (spec-spec_mean)/(spec_std+eps)
        spec = np.nan_to_num(spec, nan=0.0)
        
        #If data augmentation
        if self.aug:

            eeg = self.transforms(image=eeg)["image"]
            spec = self.transforms(image=spec)["image"]

        #Convert to tensors
        eeg = torch.tensor(eeg.copy())
        spec = torch.tensor(spec.copy())

        return eeg, spec


    def __getitem__(self, index) -> dict:
    
        row = self.df.iloc[index]

        eeg_id = row["eeg_id"]
        spec_id = row["spectrogram_id"]

        #EEG Sub-sampling
        start = int(row["eeg_label_offset_seconds"]*self.eeg_sample_freq)
        end = int((row["eeg_label_offset_seconds"]+50)*self.eeg_sample_freq)
        eeg = all_eeg[eeg_id][start:end].T
        
        #Spectrogram Sub-sampling
        start = int(row["spectrogram_label_offset_seconds"]*self.spec_sample_freq)
        end = int((row["spectrogram_label_offset_seconds"]+600)*self.spec_sample_freq)
        spec = all_spec[spec_id][start:end].T

        #Normalizing and getting rid of Nans
        eeg, spec = self.format_data(eeg, spec)

        #Convert label to tensor
        label = torch.tensor(row[label_cols], dtype=torch.float32)

        return eeg, spec, label


    def __len__(self):

        return self.df.shape[0]

    @staticmethod
    def collate_fn(batch):

        eeg, spec, label = zip(*batch)
        
        if eeg is not None:
            eeg = torch.stack(eeg, dim=0).float()
        
        if spec is not None:
            spec = torch.stack(spec, dim=0).float().unsqueeze(1).expand(-1,3,-1,-1)

        if label is not None:            
            label = torch.stack(label, dim=0)
        
        return {
            "eeg": eeg,
            "spec": spec,
            "label": label
        }
    

In [8]:
#Check dataset object is working

test_dataset = HMSDataset(df=test_df)

print([data.shape for data in test_dataset[0]])

print(test_dataset[0])

[torch.Size([19, 10000]), torch.Size([400, 300]), torch.Size([6])]
(tensor([[-0.6578, -0.6246, -0.6043,  ...,  2.1034,  1.9578,  2.1060],
        [-0.9152, -0.8733, -0.8920,  ...,  1.6163,  1.5519,  1.6318],
        [-0.5338, -0.5083, -0.5434,  ...,  1.8302,  1.7877,  1.8650],
        ...,
        [-0.4207, -0.4307, -0.5225,  ...,  1.9346,  1.9449,  1.9719],
        [-0.8559, -0.8398, -0.8708,  ...,  1.6549,  1.6356,  1.6743],
        [-0.1548, -0.1980, -0.2424,  ..., -0.2740, -0.3445, -0.3278]]), tensor([[ 1.4396,  1.1608,  1.2323,  ...,  0.6481,  0.6606,  0.6204],
        [ 1.5873,  1.3349,  1.2715,  ...,  0.9713,  0.7645,  0.8367],
        [ 1.6126,  1.3039,  1.3442,  ...,  1.1260,  1.1426,  1.0092],
        ...,
        [-1.9244, -2.1493, -2.1493,  ..., -0.0883, -0.2433,  0.1278],
        [-1.7645, -1.9244, -2.1493,  ..., -0.1020, -0.2085,  0.1044],
        [-1.7645, -1.9244, -2.1493,  ...,  0.1233, -0.2525,  0.3166]]), tensor([0., 0., 0., 1., 0., 0.]))


### Model

Load your model here, the current models are supported:

1. CNN
    - ConvNext
    - EfficientNet b0
    - EfficientNet v2s
2. Custom Architecture
    - Contrastive Detector

In [9]:
model_config = {
    "model_name": "efficientnet_b0",
    "num_features": 512,
    "num_classes": 6,
}

# model = CNNDetector(model_config).to(device)
model = ContrastiveDetector(model_config).to(device)

ContrastiveDetector(
  (kl_loss): KLDivLoss()
  (backbone_eeg): Sequential(
    (0): Conv1d(19, 64, kernel_size=(5,), stride=(5,), bias=False)
    (1): BatchNorm1d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
    (2): ReLU()
    (3): Conv1d(64, 128, kernel_size=(7,), stride=(5,), bias=False)
    (4): Sequential(
      (0): BatchNorm1d(128, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (1): ReLU()
      (2): Conv1d(128, 128, kernel_size=(5,), stride=(3,), bias=False)
    )
    (5): Sequential(
      (0): BatchNorm1d(128, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (1): ReLU()
      (2): Conv1d(128, 128, kernel_size=(5,), stride=(3,), bias=False)
    )
    (6): Sequential(
      (0): BatchNorm1d(128, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (1): ReLU()
      (2): Conv1d(128, 128, kernel_size=(5,), stride=(3,), bias=False)
    )
    (7): Sequential(
      (0): BatchNorm1d(128, eps=1e-05, 

### Training and Testing

Validation is done every `test_step` to check if model is overfitting on training data.

Testing is done at the end of every epoch with the model produced by that epoch.

Training and testing metrics can be viewed in tensorboard as mentioned in the README.

In [10]:
@torch.no_grad()
def validate(model, valid_dataloader):

    model.eval()

    loss_fn = nn.KLDivLoss(reduction="batchmean")

    loss = torch.tensor([0.]).to(device)

    for batch in tqdm(valid_dataloader):

        for key in batch:

            batch[key] = batch[key].to(device)

        predictions = model(batch)
        
        predictions = F.log_softmax(predictions,dim=1)

        loss += loss_fn(predictions,batch["label"])

    loss = loss / len(valid_dataloader)

    model.train()

    print("Validation loss", loss.item())

    return loss.item()

@torch.no_grad()
def test(model, test_dataloader):

    model.eval()

    loss_fn = nn.KLDivLoss(reduction="batchmean")

    num_classes = 6

    acc = MulticlassAccuracy(average="macro", num_classes=num_classes)
    f1_score = torch.tensor([0.]).to(device)
    loss = torch.tensor([0.]).to(device)
     
    for batch in tqdm(test_dataloader):

        for key in batch:
            batch[key] = batch[key].to(device)

        predictions = model(batch)

        loss += loss_fn(F.log_softmax(predictions, dim=1), batch["label"])

        predictions = F.softmax(predictions, dim=1)

        predictions = torch.argmax(predictions, dim=1)

        classes = torch.argmax(batch["label"], dim=1)

        f1_score += multiclass_f1_score(predictions, classes, num_classes=num_classes)

        acc.update(predictions, classes)

    model.train()

    f1_score /= len(test_dataloader)
    loss /= len(test_dataloader)

    return acc.compute(), f1_score, loss

#training

def train_epoch(model=None,
          train_dataloader=None,
          valid_dataloader=None,
          test_dataloader=None,
          optimiser = None,
          train_config=None,
          valid_config=None,
          lr_scheduler=None,
          min_valid_loss=100.,
          min_test_loss=100.,
          model_name="model",
          epoch=0,
          logger=None):
    
    """
    Training Function
    """

    assert(train_dataloader is not None)
    
    assert(model is not None)

    #Training Params

    save_model = train_config.get("save_model", True)
    to_model_keys = ["spec", "eeg", "label"]
    valid_step = valid_config.get("valid_step", 1000)
    verbose_step = train_config.get("verbose_step", 10)
    
    
    loss_fn = nn.KLDivLoss(reduction="batchmean")
    
    model.train()
    
    for itr, batch in tqdm(enumerate(train_dataloader)):

#       print(batch["eeg"].shape)

        #Train One Iteration

        #Load data to GPU

        for key in to_model_keys:

            batch[key] = batch[key].to(device)

        predictions = model(batch)

        if "contrastive" in model_name:

            klloss, contrastive_loss = model.get_loss(predictions, batch["label"])

            loss = klloss+contrastive_loss
        
        else:
            
            loss = model.get_loss(predictions,batch["label"])
        
        optimiser.zero_grad()

        loss.backward()

        optimiser.step()
        
        if lr_scheduler is not None:
            lr_scheduler.step()
        
        if itr%verbose_step==0:
            print(f"Training itr {itr}/{len(train_dataloader)}")
            for param_group in optimiser.param_groups:
                lr = param_group['lr']
                break
            

            if "contrastive" in model_name:
                print("Training KL Loss: ", klloss.item(), "Training Contrastive Loss: ", contrastive_loss.item(), "Learning Rate:", lr)
                logger.log_metrics({"train/klloss":klloss, "train/contrastiveloss": contrastive_loss, "train/lr": lr}, itr+epoch*len(train_dataloader))
            else:
                print("Training Loss: ", loss.item(), "Learning Rate:", lr)
                logger.log_metrics({"train/loss":loss, "train/lr": lr}, itr+epoch*len(train_dataloader))
        
        #Validation
        
        if itr%valid_step==0 and itr>0 and valid_dataloader is not None:
            print("Validation")
            valid_loss = validate(model, valid_dataloader)
            
            if valid_loss < min_valid_loss and save_model:
                min_valid_loss = valid_loss
                save_path = os.path.join(DATA_ROOT,"models")
                os.makedirs(save_path, exist_ok=True)
                torch.save(model.state_dict(), os.path.join(save_path, f"{model_name}_best.pt"))
            logger.log_metrics({"valid/loss":valid_loss}, itr+epoch*len(train_dataloader))
        
    #Test
    if test_dataloader is not None:
        acc, f1_score, loss = test(model, test_dataloader)
        if loss < min_test_loss:
            min_test_loss = loss
        logger.log_metrics({"test/acc":acc, "test/f1_score":f1_score, "test/loss":loss}, itr+epoch*len(train_dataloader))
    return min_valid_loss, min_test_loss

In [11]:
def train(model=None,
          optimiser=None,
          lr_scheduler=None,
          train_config=None,
          valid_config=None,
          test_dataloader=None,
          model_name="model",
          logger=None):
    
    train_batch_size = train_config.get("batch_size", 32)
    num_epochs = train_config.get("num_epochs", 10)
    valid_folds = train_config.get("valid_folds", 5)
    valid_batch_size = valid_config.get("batch_size", 32)
    train_workers = train_config.get("workers", 1)
    valid_workers = valid_config.get("workers", 1)

    min_valid_loss = 100.
    min_test_loss = 100.

    for epoch in range(num_epochs):

        print("Epoch", epoch)

        #KGFolds
        fold = np.random.randint(valid_folds)
        train_df_fold = train_df[train_df["fold"]!=fold]
        valid_df = train_df[train_df["fold"]==fold]
        train_dataset = HMSDataset(train_df_fold, aug=True)
        valid_dataset = HMSDataset(valid_df, aug=False)

        train_dataloader = DataLoader(
            dataset=train_dataset,
            batch_size = train_batch_size,
            shuffle=True,
            num_workers=train_workers,
            collate_fn=train_dataset.collate_fn
        )

        valid_dataloader = DataLoader(
            dataset=valid_dataset,
            batch_size = valid_batch_size,
            shuffle=True,
            num_workers=valid_workers,
            collate_fn=valid_dataset.collate_fn
        )

        min_valid_loss, min_test_loss = train_epoch(model=model,
                    train_dataloader=train_dataloader,
                    valid_dataloader=valid_dataloader,
                    test_dataloader=test_dataloader,
                    optimiser=optimiser,
                    train_config=train_config,
                    valid_config=valid_config,
                    lr_scheduler=lr_scheduler,
                    min_valid_loss=min_valid_loss,
                    min_test_loss=min_test_loss,
                    model_name=model_name,
                    epoch=epoch,
                    logger=logger)
        print("Min Valid Loss", min_valid_loss, "Min Test Loss", min_test_loss)

#### Hyperparameters

Configuration of hyperparamters for training and validation/testing is shown below.

If you would like to train the contrastive detector, set `model_name` to "contrastive_{backbone_name}", e.g. "contrastive_efficientnetb0". Otherwise, Set `model_name` to "cnn_detector_{backbone_name}".

- `batch_size`: size of one mini-batch of data per forward pass of the model during training, validation and testing respectively
- `lr`: learning rate, depends on optimiser and learning rate scheduler
- `save_model`: saves the best model determined by validation loss
- `valid_folds`: set during dataset geneartion, the number of groups to have in Group KFolds training
- `verbose_step`: iteration step interval to print and log training metrics
- `valid_step`: iteration step interval to perform validation on model and save best model by validation loss
- `workers`: number of CPU workers to load dataset for training, validation, and testing. A higher number requires more RAM but may speed up training

In [12]:
train_config = {
    "batch_size": 32,
    "num_epochs": 5,
    "lr": 1e-3,
    "save_model": True,
    "workers": 1,
    "verbose_step": 100,
    "weight_decay": 1e-4,
    "valid_folds": VALID_FOLDS,
}

valid_config = {
    "batch_size": 32,
    "workers": 1,
    "valid_step": 500
}

lr = train_config.get("lr", 1e-3)
weight_decay = train_config.get("weight_decay", 0.)
model_name = "contrastive"
num_epochs = train_config.get("num_epochs", 1)
folds = train_config.get("valid_folds", 5)
train_batch_size = train_config.get("batch_size", 32)
test_batch_size = valid_config.get("batch_size", 32)


logger = TensorBoardLogger(f"logs/{model_name}", name=model_name)

optimiser = torch.optim.Adam(
model.parameters(),
lr=lr,
weight_decay=weight_decay
)

lr_scheduler = torch.optim.lr_scheduler.OneCycleLR(
    optimiser,
    max_lr=lr,
    epochs=num_epochs,
    steps_per_epoch=int((folds-1)/folds*train_df.shape[0]/train_batch_size)+10
)

test_dataset = HMSDataset(df=test_df)

test_dataloader = DataLoader(
    dataset=test_dataset,
    batch_size=test_batch_size,
    collate_fn=test_dataset.collate_fn
)

torch.cuda.empty_cache()

train(model=model,
      train_config=train_config,
      valid_config=valid_config,
      test_dataloader=test_dataloader,
      model_name=model_name,
      optimiser=optimiser,
      lr_scheduler=lr_scheduler,
      logger=logger)

    

Epoch 0


0it [00:00, ?it/s]

Training itr 0/1202
Training KL Loss:  1.535893440246582 Training Contrastive Loss:  -0.010634609498083591 Learning Rate: 4.0000718651559374e-05


/home/benluo/miniconda3/envs/DeepfakeBench/lib/python3.7/site-packages/numpy/core/fromnumeric.py:86: RuntimeWarning: invalid value encountered in reduce
  return ufunc.reduce(obj, axis, dtype, out, **passkwargs)
/home/benluo/miniconda3/envs/DeepfakeBench/lib/python3.7/site-packages/numpy/lib/nanfunctions.py:1545: RuntimeWarning: invalid value encountered in subtract
  np.subtract(arr, avg, out=arr, casting='unsafe')
/home/benluo/miniconda3/envs/DeepfakeBench/lib/python3.7/site-packages/ipykernel_launcher.py:34: RuntimeWarning: invalid value encountered in subtract


Training itr 100/1202
Training KL Loss:  0.8607556819915771 Training Contrastive Loss:  0.0014440745580941439 Learning Rate: 4.731232458808946e-05
Training itr 200/1202
Training KL Loss:  0.6186394095420837 Training Contrastive Loss:  -0.00841532088816166 Learning Rate: 6.874272265430651e-05
Training itr 300/1202
Training KL Loss:  0.7805444598197937 Training Contrastive Loss:  -0.03355320543050766 Learning Rate: 0.00010365180448417455
Training itr 400/1202
Training KL Loss:  0.6708669662475586 Training Contrastive Loss:  -0.142753005027771 Learning Rate: 0.00015099686451240733
Training itr 500/1202
Training KL Loss:  0.486168771982193 Training Contrastive Loss:  -0.269393652677536 Learning Rate: 0.0002093637447321907
Validation


  0%|          | 0/301 [00:00<?, ?it/s]

Validation loss 0.8027375936508179


╭─────────────────────────────── Traceback (most recent call last) ────────────────────────────────╮
│ in <module>:59                                                                                   │
│ in train:59                                                                                      │
│ in train_epoch:133                                                                               │
│                                                                                                  │
│ /home/benluo/miniconda3/envs/DeepfakeBench/lib/python3.7/site-packages/torch/optim/lr_scheduler. │
│ py:65 in wrapper                                                                                 │
│                                                                                                  │
│     62 │   │   │   │   instance = instance_ref()                                                 │
│     63 │   │   │   │   instance._step_count += 1                                                 │
│     64 │   │   │   │   wrapped = func.__get__(instance, cls)                                     │
│ ❱   65 │   │   │   │   return wrapped(*args, **kwargs)                                           │
│     66 │   │   │                                                                                 │
│     67 │   │   │   # Note that the returned function here is no longer a bound method,           │
│     68 │   │   │   # so attributes like `__func__` and `__self__` no longer exist.               │
│                                                                                                  │
│ /home/benluo/miniconda3/envs/DeepfakeBench/lib/python3.7/site-packages/torch/optim/optimizer.py: │
│ 109 in wrapper                                                                                   │
│                                                                                                  │
│   106 │   │   │   │   obj, *_ = args                                                             │
│   107 │   │   │   │   profile_name = "Optimizer.step#{}.step".format(obj.__class__.__name__)     │
│   108 │   │   │   │   with torch.autograd.profiler.record_function(profile_name):                │
│ ❱ 109 │   │   │   │   │   return func(*args, **kwargs)                                           │
│   110 │   │   │   return wrapper                                                                 │
│   111 │   │                                                                                      │
│   112 │   │   hooked = getattr(self.__class__.step, "hooked", None)                              │
│                                                                                                  │
│ /home/benluo/miniconda3/envs/DeepfakeBench/lib/python3.7/site-packages/torch/autograd/grad_mode. │
│ py:27 in decorate_context                                                                        │
│                                                                                                  │
│    24 │   │   @functools.wraps(func)                                                             │
│    25 │   │   def decorate_context(*args, **kwargs):                                             │
│    26 │   │   │   with self.clone():                                                             │
│ ❱  27 │   │   │   │   return func(*args, **kwargs)                                               │
│    28 │   │   return cast(F, decorate_context)                                                   │
│    29 │                                                                                          │
│    30 │   def _wrap_generator(self, func):                                                       │
│                                                                                                  │
│ /home/benluo/miniconda3/envs/DeepfakeBench/lib/python3.7/site-packages/torch/optim/adam.py:171   │
│ in step                                                    